In [225]:
import time
import numpy as np
from numba import int32, float32, vectorize, njit, guvectorize, float64, prange, intp  # import the types
from numba.experimental import jitclass

In [226]:
def gu_random_normal(func, *args, **kwargs):
    """
    The guvectorize decorated method, runs in nopython mode
    :param func: a loss error method
    :param target: choose between | 1: None -> serial | 2: 'parallel' -> parallel | execution method
    :return: the decorated loss, error function
    """
    kwargs_ = {k: v for k, v in kwargs.items() if v is not None}
    return guvectorize([(float32, float32, float32[:,:], float32[:,:]),
              (float64, float64, float64[:,:], float64[:,:])],'(),(),(m,n)->(m,n)', nopython=True, fastmath=True, *args, **kwargs_)(func)

def random_normal(mu, sigma, arr, out):
    dim1 = arr[0].shape
    dim2 = arr[1].shape
    for i in prange(*dim1):
        for j in prange(*dim2):
            out[i][j] = np.random.normal(mu, sigma)

In [227]:
shape = np.zeros((2000,2000), dtype=np.float32)
s_rnd_norm = gu_random_normal(random_normal)
p_rnd_norm = gu_random_normal(random_normal, target='parallel')

In [228]:
def ntime(func1):
    start = time.time()
    func1(0, 2, shape)
    end = time.time()
    print("Elapsed Numba Pre = %s" % (end - start))
    start = time.time()
    f2 = func1(0, 2, shape)
    end = time.time()
    print("Elapsed Numba Post = %s" % (end - start))
    return f2

def ptime():
    start = time.time()
    f1 = np.random.normal(0, 2, (20000, 20000))
    end = time.time()
    print("Elapsed PyTime = %s" % (end - start))
    return f1

In [229]:
ntime(s_rnd_norm)

Elapsed Numba Pre = 0.0828244686126709
Elapsed Numba Post = 0.0807652473449707


array([[ 1.2723088 ,  1.3387862 , -0.48738942, ..., -0.83457154,
         1.8766505 , -1.7317517 ],
       [ 0.36621222, -0.70813745, -0.1992119 , ...,  0.33104214,
         0.16246265,  0.16894118],
       [-1.7813585 , -0.2928216 , -1.3698263 , ..., -0.70905495,
         2.0705454 , -0.4166943 ],
       ...,
       [ 1.0389138 , -0.50361574, -1.958488  , ..., -4.8322167 ,
         1.5135796 , -0.21287723],
       [-0.76075965,  0.49102527, -0.8238306 , ..., -0.39448524,
        -0.50381696, -0.5980589 ],
       [ 0.14896476, -0.86886746,  1.0353479 , ...,  1.8339335 ,
         1.5804843 ,  2.1500037 ]], dtype=float32)

In [230]:
ntime(p_rnd_norm)

Elapsed Numba Pre = 0.08176612854003906
Elapsed Numba Post = 0.0797872543334961


array([[-2.1440723 , -2.2859163 , -3.1708097 , ..., -3.9074945 ,
        -0.45936233,  2.7470052 ],
       [ 0.9685498 , -1.5892606 , -1.3003803 , ..., -0.36321074,
         0.12206651, -2.366153  ],
       [-1.1292521 , -2.264473  ,  1.7200072 , ..., -0.4070027 ,
         4.197665  ,  0.5973122 ],
       ...,
       [-0.52780545,  0.76551336, -1.1112909 , ...,  2.0779326 ,
         0.24334511, -3.19986   ],
       [-2.3833132 , -0.5552249 , -0.53840846, ...,  0.62006795,
         2.575955  ,  1.7154983 ],
       [ 1.2176105 ,  1.1854285 ,  1.9820133 , ..., -1.0665817 ,
        -2.6381419 , -0.95294094]], dtype=float32)

In [231]:
ptime()


Elapsed PyTime = 9.08271336555481


array([[ 2.94384583,  1.4411226 ,  0.29812779, ..., -2.23757692,
        -0.10575123, -1.35847441],
       [ 0.52092974,  1.06742107, -0.55918857, ...,  2.91952741,
        -0.47194981, -3.76958197],
       [ 3.35081503, -0.79997393,  5.13770238, ...,  1.40563647,
         1.24586471, -0.53034035],
       ...,
       [ 2.08999267, -4.14367295, -1.46096909, ...,  0.18077225,
         2.12557491, -1.85438452],
       [ 3.2827488 , -3.45519774,  0.33831735, ..., -0.24976689,
         1.12164158,  0.68423069],
       [ 3.29175257,  0.26550488,  3.00913531, ..., -1.05405684,
         4.12808288, -1.8439115 ]])